# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch

In [10]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [11]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [12]:
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [13]:
index_name = 'mohamad_index1'
es = Elasticsearch("http://localhost:9200")
es.indices.delete(index = index_name)
es.indices.create(index = index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mohamad_index1'})

### Check the Cluster Status

In [14]:
dict(es.info())

{'name': 'Mamadss',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'XXMhwF3YTAyZ0r2DJ8yDgw',
 'version': {'number': '8.3.0',
  'build_type': 'tar',
  'build_hash': '5b8b981647acdf1ba1d88751646b49d1b461b4cc',
  'build_date': '2022-06-23T22:48:49.607492124Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [14]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [03:41<00:00, 55.02it/s]


###  Bulk API

In [15]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': 'mohamad_index1',
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [16]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 7.65 seconds


### check index

In [18]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [19]:
query= {
        "bool": {
          "should": [
              { 
                  "match":{
                      "content": {
                                    "query": "آمریکا"
                          }
                  }},{
                  "match_phrase":{
                      "content": {
                                   "query": "تحریم هسته‌ای"
                          }
                  }
                  #TODO: add a match query structure ==> use for normal words
              }], 
              
          "must_not": 
              {
                  "match":{
                      "content": {
                           "query": "ایران"
                                    
                          }
                  }
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
              }
          ,
            
        },
    }

In [127]:
query= {
        "bool": {
         "must":
                {
                   "match_phrase":{
                      "content": {
                                   "query": "تحریم هسته‌ای"
                          }
                  }
                }
              
         
         
            
        },
    }

In [35]:
query= {
        "bool": {
          "should": [
              { 
                  "match":{
                      "content": {
                                    "query": "صهیونیست"
                          }
                  }}], 
              
          "must_not": 
              {
                  "match":{
                      "content": {
                           "query": "اورشلیم"
                                    
                          }
                  }
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
              }
          ,
            
        },
    }

In [36]:
print(query)

{'bool': {'should': [{'match': {'content': {'query': 'صهیونیست'}}}], 'must_not': {'match': {'content': {'query': 'اورشلیم'}}}}}


### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [37]:
res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)

In [38]:
print(res)

{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 60, 'relation': 'eq'}, 'max_score': 9.867652, 'hits': [{'_index': 'mohamad_index1', '_id': '6330', '_score': 9.867652, '_ignored': ['content.keyword'], '_source': {'url': 'https://www.farsnews.ir/news/14001002000158/محکومیت-حضور-کاروان-ورزشی-رژیم-صهیونستی-در-امارات-عکس'}}, {'_index': 'mohamad_index1', '_id': '6451', '_score': 9.743655, '_ignored': ['content.keyword'], '_source': {'url': 'https://www.farsnews.ir/news/14000929000460/موج-اعتراضات-از-ایران-به-عراق-رسید-محکومیت-یونس-محمود-مثل-مهدوی\u200cکیا'}}, {'_index': 'mohamad_index1', '_id': '6621', '_score': 9.466341, '_ignored': ['content.keyword'], '_source': {'url': 'https://www.farsnews.ir/news/14000928000230/اتفاق-عجیب-در-ادامه-عادی-سازی-روابط-رژیم-اشغالگر-با-جهان-اسلام-مهدوی'}}, {'_index': 'mohamad_index1', '_id': '6524', '_score': 9.328856, '_ignored': ['content.keyword'], '_source': {'url': 'https://

### Results

In [39]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

60 results in 0.006 s: 
https://www.farsnews.ir/news/14001002000158/محکومیت-حضور-کاروان-ورزشی-رژیم-صهیونستی-در-امارات-عکس
https://www.farsnews.ir/news/14000929000460/موج-اعتراضات-از-ایران-به-عراق-رسید-محکومیت-یونس-محمود-مثل-مهدوی‌کیا
https://www.farsnews.ir/news/14000928000230/اتفاق-عجیب-در-ادامه-عادی-سازی-روابط-رژیم-اشغالگر-با-جهان-اسلام-مهدوی
https://www.farsnews.ir/news/14000929000102/مهدوی‌کیا-حامل-پرچم-رژیم-اشغالگر-شد-خشم-صهیونیست‌ها-از-ستارگان
https://www.farsnews.ir/news/14001004000131/مهدوی‌کیا-به-جای-عذرخواهی-در-دام-صهیونیست‌ها-افتاد-سکوت-شائبه‌برانگیز
https://www.farsnews.ir/news/14001102000416/ورزشکار-کویتی-دست-رد-به-سینه-صهیونیست‌ها-زد-عکس
https://www.farsnews.ir/news/14001119000767/رونمایی-از-جایزه-خوش-خدمتی-سعودی‌ها-به-رژیم-صهیونیستی-در-جودو-عکس
https://www.farsnews.ir/news/14001006000161/اقدام-حمایتی-عمان-از-فلسطین-سرود-و-پرچم-رژیم-اشغالگر-بایکوت-شد-عکس
https://www.farsnews.ir/news/14001011000158/لابی-صهیونیست‌ها-در-ورزش-هشدار-جدی-به-مالزی-به-خاطر-حمایت-از-فلسطین
https:/